In [9]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


In [3]:
cur = db.list_collections()
for collection in cur:
    print(collection)

{'name': 'companies', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('fec41d62-1d18-46c6-b5d0-9daec6f429ba')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'companies.companies'}}


# Indentify companies with more than 10 years

In [ ]:
df = pd.DataFrame(json_normalize(old_comps,["offices"]))


In [55]:
old_comps = list(db.companies.find({"founded_year":{"$lte":2010}},{"_id":0,"name":1,"founded_year":1,"offices.address1":1,"offices.city":1,"offices.country_code":1,"offices.latitude":1,"offices.longitude":1,"offices.zip_code":1}))
dfo = json_normalize(old_comps, "offices", ["name","founded_year"])
dfo

,address1,zip_code,city,country_code,latitude,longitude,name,founded_year
0,4900 Hopyard Rd.,94588,Pleasanton,USA,37.692934,-121.904945,AdventNet,1996
1,710 - 2nd Avenue,98104,Seattle,USA,47.603122,-122.333253,Wetpaint,2005
2,270 Lafayette Street,10012,New York,USA,40.723731,-73.996431,Wetpaint,2005
3,4900 Hopyard Rd,94588,Pleasanton,USA,37.692934,-121.904945,Zoho,2005
4,Suite 200,94301,Palo Alto,ISR,NaN,NaN,Omnidrive,2005
...,...,...,...,...,...,...,...,...
13979,P.O. Box 2096,07039,Livingston,USA,40.793024,-74.323554,AfterLogic,2002
13980,Near Tube Well(south),334003,Bikaner,IND,NaN,NaN,Adhunk,2009
13981,200 Knickerbocker Avenue,11716,Bohemia,USA,40.775055,-73.088140,Contemporary Computer Services,1974
13982,400 Royal Palm Way,33480,Palm Beach,USA,26.705331,-80.041395,QSGI,1967


In [56]:
dfo.isnull().sum()

address1         109
zip_code         100
city              35
country_code       0
latitude        4910
longitude       4910
name               0
founded_year       0
dtype: int64

In [57]:
dfo.zip_code.value_counts()

              1803
94105          144
94107          136
94103           99
95054           80
              ... 
95056-4428       1
416 72           1
08753            1
1600-100         1
M5C 2K4          1
Name: zip_code, Length: 5719, dtype: int64

# Indentify tech startups which raised more than 1M

In [58]:
cool_comps = list(db.companies.find({"$and":[{"founded_year":{"$lte":2010}},{"total_money_raised":{"$not":{"$regex":"\$0|\.*k"}}}]},{"_id":0,"name":1,"total_money_raised":1,"founded_year":1,"offices.address1":1,"offices.city":1,"offices.country_code":1,"offices.latitude":1,"offices.longitude":1,"offices.zip_code":1}))
dfc = json_normalize(cool_comps, "offices", ["name","total_money_raised","founded_year"])
dfc

,address1,zip_code,city,country_code,latitude,longitude,name,total_money_raised,founded_year
0,710 - 2nd Avenue,98104,Seattle,USA,47.603122,-122.333253,Wetpaint,$39.8M,2005
1,270 Lafayette Street,10012,New York,USA,40.723731,-73.996431,Wetpaint,$39.8M,2005
2,9229 W. Sunset Blvd.,90069,West Hollywood,USA,34.090368,-118.393064,Geni,$16.5M,2006
3,None,None,Menlo Park,USA,37.484130,-122.169472,Gizmoz,$18.1M,2003
4,16935 W. Bernardo Dr. Suite 101,92127,San Diego,USA,33.022176,-117.081406,Slacker,$73.1M,2006
...,...,...,...,...,...,...,...,...,...
4560,905 Calle Amanecer,92673,San Clemente,USA,33.448010,-117.607717,Cameron Health,$121M,2001
4561,"2635 North 1st Street, Suite 150",95134,San Jose,USA,37.386417,-121.929170,Celestial Semiconductor,$2M,2004
4562,"Rue des Tanneurs, 60a",1000,Brussels,BEL,50.839970,4.346472,Getyoo,€1.13M,2009
4563,11 Eggleston view,DL3 9SH,London,GBR,NaN,NaN,OfficialVirtualDJ,€25M,2009


In [60]:
dfc.isnull().sum()

address1                45
zip_code                48
city                    11
country_code             0
latitude              1653
longitude             1653
name                     0
total_money_raised       0
founded_year             0
dtype: int64

In [63]:
dfc.city.value_counts()

San Francisco      396
New York           315
London             178
Seattle            103
Mountain View       92
                  ... 
Tacoma               1
Woodridge            1
Chagrin Falls        1
Cingapore            1
Fleet Hampshire      1
Name: city, Length: 995, dtype: int64